In [82]:
from itertools import combinations,product
import numpy as np
import random 
import copy 
import pandas as pd 
from enum import Enum

In [83]:
#must be in dataset, if they change the tables of availibility must get droped 
max_morning_periods = 3
max_afternoon_periods = 3
max_periods_day = 5
algorithm = 1 # 1 2 3 4 

In [84]:
import sqlite3
import pandas as pd
from sqlite3 import Error


def read_availibility():
    conn = sqlite3.connect('availability.db')
    cur = conn.cursor()

    # Create rooms_availability table if it doesn't exist
    cur.execute('''
    CREATE TABLE IF NOT EXISTS rooms_availability (
        room_id TEXT PRIMARY KEY,
        availability TEXT
    )
    ''')

    # Create teachers_availability table if it doesn't exist
    cur.execute('''
    CREATE TABLE IF NOT EXISTS teachers_availability (
        teacher_id TEXT PRIMARY KEY,
        availability TEXT
    )
    ''')

    rooms_availability = {}
    teachers_availability = {}

    cur.execute('SELECT room_id, availability FROM rooms_availability')
    rows = cur.fetchall()
    for row in rows:
        room_id, availability_str = row
        availability_array = np.array(list(map(lambda x: x == '1', availability_str.split(','))))
        rooms_availability[room_id] = availability_array

    cur.execute('SELECT teacher_id, availability FROM teachers_availability')
    rows = cur.fetchall()
    for row in rows:
        teacher_id, availability_str = row
        availability_array = np.array(list(map(lambda x: x == '1', availability_str.split(','))))
        teachers_availability[teacher_id] = availability_array

    conn.close()

    return rooms_availability, teachers_availability

    return rooms_availability, teachers_availability


def read_database(db_path, table_name):
    try:
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()

        # Fetch column names
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = [col[1] for col in cursor.fetchall()]

        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        cursor.close()
        connection.close()

        return columns, rows
    except Error as e:
        print(f"An error occurred: {e}")
        return [], []

table_names = ['rooms', 'working_days', 'teachers', 'subjects', 'groups', 'groups_teachers', 'sessions']

rooms_availability = {}
teachers_availability = {}

rooms_availability,teachers_availability = read_availibility()

dfs = {}

for table_name in table_names:
    columns, data = read_database('timetableDB.db', table_name)
    df = pd.DataFrame(data, columns=columns)
    dfs[table_name] = df



In [85]:
dfs["groups_teachers"]["session_id"] = []

In [116]:
# session type , teacher , length , student capacity, subject
# room type, capacity
# teacher name , field , priority yes no 
# constraint priority yes no 
# working days 

class Session:
    def __init__(self, session_type,subject, teacher=None, length=1,section=1,times=1,group=None,per_week=1,semester=None,year=None):
        self.session_type = session_type
        self.teacher = teacher
        self.length = length
        self.subject = subject
        self.times = 1
        self.semester = semester
        self.year = year 
        self.group = group # 0 mean iime Series Analysis and Classification lab a section
        self.section = section
        self.room = None #get assinged later in the generation
        self.per_week = per_week
    def __eq__(self, other):
        return (isinstance(other, Session) and
                self.session_type == other.session_type and
                self.teacher == other.teacher and
                self.group == other.group and
                self.section == other.section and
                self.length == other.length and
                self.subject == other.subject)
    def __hash__(self):
        return hash((self.session_type, self.teacher, self.length, self.subject, self.section, self.group))

class Room:
    def __init__(self,id, room_type, capacity,available=[]):
        self.room_type = room_type
        self.id = id
        self.capacity = capacity
        self.available = available if available.all()!=None else [True for _ in range(max_periods_day * len(working_days))]

    def __eq__(self, other):
        return (isinstance(other, Room) and
                self.room_type == other.room_type and
                self.capacity == other.capacity)
    def __hash__(self):
        return hash((self.id))
class Teacher:
    def __init__(self, id, name, field, priority,prefered_period=None,prefered_hours=[],prefered_days=[],available=[]):
        self.id = id 
        self.name = name
        self.field = field
        self.priority = priority
        self.prefered_days = prefered_days #must be from 'working_days'
        self.prefered_period = prefered_period # m : morning, f:afternoon, None:anything is fine
        self.available = available if available.all()!=None else [True for _ in range(max_periods_day * len(working_days))]
    def __hash__(self):
        return hash((self.id))
    def __eq__(self, other):
        return (isinstance(other, Teacher) and
                self.id == other.id)
class Constraint:
    def __init__(self, priority):
        self.priority = priority

class Period:
    def __init__(self, teacher, session, room, time):
        self.teacher = teacher
        self.session = session
        self.room = room
        self.time = time

    def __eq__(self, other):
        return (isinstance(other, Period) and
                self.teacher == other.teacher and
                self.session == other.session and
                self.room == other.room and
                self.time == other.time)


In [117]:
sessions_table = pd.merge(dfs["sessions"], dfs["subjects"], on="subject_id")

In [118]:
import pandas as pd


data = {
    "group_id": list(dfs["groups"]["group_id"])[:20],
    "session_id": sessions_table['session_id'][:20],
    "teacher_id": dfs["teachers"]["teacher_id"][:20]
}

new_data_df = pd.DataFrame.from_dict(data)

dfs["groups_teachers"] = pd.concat([new_data_df], ignore_index=True)



In [119]:
n_years = len(dfs["groups"]["group_year"].unique())

year_section_group_dict = {
    i + 1: dfs["groups"][dfs["groups"]["group_year"] == i + 1]['group_section'].unique().tolist() 
    for i in range(n_years)
}


mappings = {
    'Sunday':1, 'Wednesday':4, 'Tuesday':3, 'Thursday':5, 'Monday':2
}

working_days =  [ mappings[i] for i in dfs["working_days"]['week_day'].unique() ] 



In [120]:
year_section_group_dict = {}
for i in range(n_years):
    year_section_group_dict[i+1] = {}
    for s in dfs["groups"][dfs["groups"]["group_year"] == i + 1]['group_section'].unique().tolist() :
        groups_for_year_section = dfs["groups"][(dfs["groups"]["group_year"] == i + 1) & (dfs["groups"]['group_section'] == s)]
        year_section_group_dict[i+1][s] = groups_for_year_section["group_id"].tolist()

section_group_year_dict = {}

for year, sections in year_section_group_dict.items():
    for section in sections:
        groups = year_section_group_dict[year][section]
        if section not in section_group_year_dict:
            section_group_year_dict[section] = {}
        section_group_year_dict[section][year] = groups


In [121]:
sessions_table["nb_of_sessions_per_week"]=sessions_table["nb_of_sessions_per_week"].apply(lambda x:int(x))

In [122]:

teacher_set = {row['teacher_id']: Teacher(
                    id=row['teacher_id'],
                    name=row['teacher_name'],
                    field=row['field'],
                    available= teachers_availability[row['teacher_id']] if row['teacher_id'] in teachers_availability else [],
                    priority=row['priority'],  
                    prefered_period=row["pereferred_period"],
                    prefered_days=[mappings[day] for day in dfs["working_days"][dfs["working_days"]["teacher_id"] == row['teacher_id']]["week_day"]]
                ) for index, row in dfs["teachers"].iterrows()}

In [123]:
group_id_to_number_map = {row["group_id"]: row["group_number"] for index, row in dfs["groups"].iterrows()}
groups_teachers = {}
for g in dfs["groups_teachers"]["group_id"]:
        groups_teachers[g] = {}
        for index,row in dfs["groups_teachers"][dfs["groups_teachers"]["group_id"] == g].iterrows():
            groups_teachers[g][row['session_id']] = teacher_set.get(row["teacher_id"]) 
 

In [124]:

sessions = {}
for index, row in sessions_table.iterrows():
    session_prof_by_group = dfs["groups_teachers"][dfs["groups_teachers"]["session_id"] == row["session_id"]]
    year_key = row["year"]
    section_dict = year_section_group_dict[year_key]
    if year_key not in sessions:
        sessions[year_key] = {}
    for section_key, group_list in section_dict.items():
        if not section_key  in sessions[year_key]:
            sessions[year_key][section_key] = []
        if row['session_type']=="lecture":      
            if year_section_group_dict[year_key][section_key][0] in groups_teachers.keys():     
                t = groups_teachers.get(year_section_group_dict[year_key][section_key][0]).get(row['session_id'])
            else :
                t = None
            sessions[year_key][section_key].append(Session(
                session_type=row['session_type'],
                subject=row['subject_title'],
                per_week=row["nb_of_sessions_per_week"],
                group=None,
                section=section_key,
                year=year_key,
                teacher=t
            ))
        else :
            for group in group_list:
                for _ in range(row["nb_of_sessions_per_week"]):
                    if group in groups_teachers.keys():
                        t = groups_teachers.get(group).get(row['session_id'])
                    else :
                        t = None
                    sessions[year_key][section_key].append(Session(
                        session_type=row['session_type'],
                        subject=row['subject_title'],
                        per_week=row["nb_of_sessions_per_week"],
                        group=group_id_to_number_map[group],
                        section=section_key,
                        year=year_key,
                        teacher=t
                    ))
available_rooms = [Room(row['room_id'], row['session_type'], row['room_capacity'], available= rooms_availability[row['room_id']] if row['room_id'] in rooms_availability else []) for index, row in dfs["rooms"].iterrows()]

In [125]:
year_section_group_dict = {}
for i in range(n_years):
    year_section_group_dict[i+1] = {}
    for s in dfs["groups"][dfs["groups"]["group_year"] == i + 1]['group_section'].unique().tolist() :
        groups_for_year_section = dfs["groups"][(dfs["groups"]["group_year"] == i + 1) & (dfs["groups"]['group_section'] == s)]
        year_section_group_dict[i+1][s] = groups_for_year_section["group_number"].tolist()

section_group_year_dict = {}

for year, sections in year_section_group_dict.items():
    for section in sections:
        groups = year_section_group_dict[year][section]
        if section not in section_group_year_dict:
            section_group_year_dict[section] = {}
        section_group_year_dict[section][year] = groups


In [128]:
group_id_to_number_map = {row["group_id"]: row["group_number"] for index, row in dfs["groups"].iterrows()}
groups_teachers = {}
for g in dfs["groups_teachers"]["group_id"]:
        groups_teachers[group_id_to_number_map[g]] = {}
        for index,row in dfs["groups_teachers"][dfs["groups_teachers"]["group_id"] == g].iterrows():
            groups_teachers[group_id_to_number_map[g]][row['session_id']] = teacher_set.get(row["teacher_id"]) 
 

In [129]:
def mergeSort(arr,f_score,way="asc"):
    if len(arr) > 1:
        mid_indx = len(arr)//2
        left = arr[:mid_indx]
        right = arr[mid_indx:]
        f_left = f_score[:mid_indx]
        f_right = f_score[mid_indx:]
        mergeSort(left,f_left,way)
        mergeSort(right,f_right,way)
        i = j = k = 0
        if way == "dsc":
            while i < len(left) and j < len(right):
                if f_left[i] <= f_right[j]:
                    f_score[k] = f_left[i]
                    arr[k] = left[i]
                    i += 1
                else:
                    f_score[k] = f_right[j]
                    arr[k] = right[j]
                    j += 1
                k += 1
        elif  way == "asc":
            while i < len(left) and j < len(right):
                if f_left[i] >= f_right[j]:
                    f_score[k] = f_left[i]
                    arr[k] = left[i]
                    i += 1
                else:
                    f_score[k] = f_right[j]
                    arr[k] = right[j]
                    j += 1
                k += 1
 
        while i < len(left):
            f_score[k] = f_left[i]
            arr[k] = left[i]
            i += 1
            k += 1
 
        while j < len(right):
            f_score[k] = f_right[j]
            arr[k] = right[j]
            j += 1
            k += 1
    return arr,f_score

In [130]:
class Generator:
    
    def __init__(self,sessions,teachers,rooms,n_sections,n_groups,working_days,max_morning_periods,max_afternoon_periods,max_periods_day,section_group_dict,perecent=50):
        self.sessions = sessions
        # order by priority 
        self.teachers = sorted(teachers, key=lambda x: x.priority)
        self.rooms = rooms
        self.n_groups = n_groups
        self.n_sections = n_sections
        self.working_days = working_days  
        self.rests = ["f"] * (n_groups//(100//perecent) )
        self.rests += ["m"] * (n_groups//(100//(100-perecent))) # extend for more then 2 sessions : perecent is a list 
        random.shuffle(self.rests)
        self.max_morning_periods = max_morning_periods
        self.max_afternoon_periods = max_afternoon_periods
        self.max_periods_day = max_periods_day
        self.section_group_dict = section_group_dict
        self.group_section_dict = {}
        for key, values in section_group_dict.items():
            for value in values:
                if value in self.group_section_dict:
                    self.group_section_dict[value].append(key)
                else:
                    self.group_section_dict[value] = [key]
        
    def generate_combination_brute(self):
        # now shuffle and stack to generate 1000 possible child 
        sessions = copy.deepcopy(self.sessions)
        random.shuffle(sessions)
        possibilities = []
        
        while len(sessions2) > 0:
            period = []
            visited_groups = []
            visited_teachers = []
            for session in sessions[:]:  
                if len(period) == 6:
                    break
                if session.group not in visited_groups and session.teacher not in visited_teachers:
                    period.append(session)
                    visited_teachers.append(session.teacher)
                    visited_groups.append(session.group)
                    sessions.remove(session)  
            
            random.shuffle(period)
            possibilities.append(period)
        return possibilities

    def generate_combination(self):
        #for each groups, gather its session { '1' : .. , '2': ....sessions of g2 , ... 
        group_combinations = {}
        for i in range (self.n_groups):
            group_combinations[i+1] = []
        for i in self.sessions:
            if i.session_type == "lecture":
                for g in self.section_group_dict[i.section]:
                    group_combinations[g] += [i]
                    break #one group is enought as its gonna get generelized later on 
            else:
                group_combinations[i.group] += [i]
                random.shuffle(group_combinations[i.group])
            
        return group_combinations

    def populate(self,n):
        population = []
        remainder = []
        for i in range(n):
            group_combinations = self.generate_combination()
            result,remain = self.propagate(group_combinations)
            population.append(result)
            remainder.append(remain)
        return population,remainder
    
    def check_teacher(self,teacher, lists,j):
        for i in lists:
            teacher_obj = next((t for t in set(self.teachers) if t and i.teacher and t.id == i.teacher.id), None)
            if teacher == i.teacher or (teacher_obj and (len(teacher_obj.available)<=j or not teacher_obj.available[j])):
                return False
        return True
    
    def check_room(self,room, list,j):
        for i in list:
            room_obj = next((t for t in set(self.rooms) if i.room and t.id == i.room.id), None)
            if room == i.room or (room_obj and not room_obj.available[j]):
                return False
        return True

    def check_group(self,group, list):
        for i in list:
            if group == i.group:
                return False
        return True

    def is_empty(self,group_combinations):
        for period in group_combinations.values():
            if period:
                return False
        return True
        
    def check_lecture(self,choice,period):
        for i in period :
            if i.session_type == "lecture" :
                if choice.group in self.section_group_dict[i.section] or choice.group == i.group or choice.section == i.section:
                    return False
            if choice.session_type == "lecture" and i.group in self.section_group_dict[choice.section] :
                return False
        return True

    def check_lecture_2(self,period):
        n_periods =  0 # must sum up to n_sections
        n_lectures = 0
        for i in period :
            if i.session_type=="lecture" :
                n_periods += len(self.section_group_dict[i.section])
                n_lectures+=1
            else :
                n_periods +=1
            if n_lectures > self.n_sections:
                return False
            if n_periods >= self.n_groups :
                    return False 
        return True

                
    def propagate(self,group_combinations, cool_down=10,max_itt = 1000):
        sl = []
        counter = 0
        while not self.is_empty(group_combinations) and max_itt>0 :
            starter = random.choice(list(group_combinations.keys()))
            if len(group_combinations[starter]) == 0:
                del group_combinations[starter]
                continue  # move to the next iteration if the selected group is empty
            period = [random.choice(group_combinations[starter])]
            group_combinations[starter].remove(period[0])
            if len(group_combinations[starter]) == 0:
                    del group_combinations[starter]
                    continue
            left_groups = [i for i in list(group_combinations.keys()) if i != starter]
            for i in left_groups:
                if len(group_combinations[i]) > 0:
                    choice = random.choice(group_combinations[i])
                    if self.check_teacher(choice.teacher, period,counter) and self.check_lecture(choice,period) and  self.check_lecture_2(period):
                        period.append(choice)
                        group_combinations[i].remove(choice)
                        if  self.check_lecture_2(period): #is full
                            continue
                        if len(group_combinations[i]) == 0 :
                                del group_combinations[i]
                                continue
                    try:
                        while not self.check_group(choice.group, period) and cool_down > 0:
                            cool_down -= 1
                            choice = random.choice(group_combinations[i])
                            if self.check_group(choice.group, period) and self.check_teacher(choice.teacher, period,counter) and self.check_lecture(choice,period) and  self.check_lecture_2(period):
                                period.append(choice)
                                group_combinations[i].remove(choice)
                                if self.check_lecture_2(period): #is full
                                    break
                                if len(group_combinations[i]) == 0:
                                    del group_combinations[i]
                    except:
                        pass
                    cool_down = 10
            sl.append(period)
            counter+=1
            if len(group_combinations[starter]) == 0:
                del group_combinations[starter]
            max_itt-=1
        return sl,group_combinations

    def is_valid(self,a,b,x,y):
        # class teacher module
        for index,i in enumerate(a) : 
            if b[y].group == i.group or b[y].teacher == i.teacher :
                return False
        for index,i in enumerate(b) : 
            if  a[x].group == i.group or a[x].teacher == i.teacher :
                return False
    
        return True
        
    def modify(self,time_table):
        max_iterations = 10000
        
        for _ in range(max_iterations):
            try:
                x = random.randint(0, len(time_table) - 1)
                y = random.randint(0, len(time_table[x]) - 1)
                u = random.randint(0, len(time_table) - 1)
                v = random.randint(0, len(time_table[u]) - 1)
            except:
                return time_table
            if  self.can_add_session(time_table[u][v],time_table[x],u) and self.can_add_session(time_table[x][y],time_table[u],x):
                time_table[x][y], time_table[u][v] = time_table[u][v], time_table[x][y]
                return time_table
    
        return time_table

    def rearange(self,time_table, max_itt,max_stuck ):
        
        best = time_table
        current = time_table
        best_score = self.heuristic(time_table)
        itt = max_itt
        stuck = max_stuck
        
        while itt > 0 :
            new = self.modify(copy.deepcopy(current))
            if self.heuristic(new) > best_score :
                best = new
                best_score = self.heuristic(new)
            if False and self.heuristic(new) == best_score and new != best:
                # new branch 
                posibility = rearange(copy.deepcopy(new),heuristic,copy.deepcopy(itt),copy.deepcopy(stuck))
                if self.heuristic(posibility) > best_score :
                    best = posibility
                    best_score = self.heuristic(posibility)
            if self.heuristic(new) >= self.heuristic(current) :
                current = new
            else :
                stuck-=1 
            if stuck == 0 :
                # allow a drop 
                stuck = max_stuck
                current = new
            itt -= 1
            
        return best

    def can_add_session(self, session, period,i):
        return self.check_group(session.group, period) and self.check_teacher(session.teacher, period,i) and self.check_lecture(session,period) and self.check_lecture_2(period)

    
    def search_2(self, time_table, max_itt=10, max_itt_2=100,r=None,rem=None):
        if r and rem :
            result = r
            remaining = rem
        else : 
            result = [[] for _ in range(self.max_periods_day * len(self.working_days))]
            remaining = [session for period in time_table for session in period]
        while len(remaining) > 0 and max_itt_2 > 0:
            temp = max_itt
            for i in range(len(result)):
                while self.check_lecture_2(result[i]) and max_itt > 0:
                    best_posibility = None
                    best_h = -1
                    for session in remaining:
                        if self.can_add_session(session, result[i],i):
                            p = copy.deepcopy(result)
                            p[i].append(session)
                            h = self.heuristic(p)
                            if h > best_h:
                                best_posibility = session
                                best_h = h
                    if best_posibility:
                        result[i].append(best_posibility)
                        remaining.remove(best_posibility)
                    max_itt -= 1
                    if max_itt == 0:
                        max_itt = temp
                        break  # Exit the inner loop if max_itt is reached
            max_itt_2 -= 1
    
        return result,remaining


    def search_3(self):
        # start fulifying constraints one by one, ex teachre with highest priority - > preference ... keep a list of constraints that did not get fulified         
        time_table =  [[] for _ in range(self.max_periods_day * len(self.working_days))]#self.init_time_table()
        sessions = copy.deepcopy(self.sessions)
        visited = []
        teachers_sessions  = {}
        for session in sessions :
            if session.teacher:
                if session.teacher not in teachers_sessions :
                    teachers_sessions[session.teacher] = [session]
                else : 
                    teachers_sessions[session.teacher].append(session)
            
        for teacher in teachers_sessions.keys(): # they are ordered by priority ( check init function )
            if teacher.prefered_days :
                for d in teacher.prefered_days:
                    d-=1
                    for session in teachers_sessions[teacher] :
                        if  session not in visited : 
                            if teacher.prefered_period == "f" : #likes morning 
                                    counter = 0
                                    time = d*(self.max_morning_periods+self.max_afternoon_periods)
                                    current_hour = i%(self.max_periods_day)
                                    mh = self.max_morning_periods - (1 if session.group and  self.rests[session.group-1] == "m" else 0)
                                    ah = self.max_afternoon_periods - (1 if  session.group and self.rests[session.group-1] == "f" else 0)
                                    while counter+mh+time<len(time_table) and counter<ah:
                                        if self.can_add_session(session, time_table[time+counter+mh],time+counter+mh):
                                            time_table[time+counter+mh].append(session)
                                            visited.append(session)
                                            break
                                        counter+=1
                            elif teacher.prefered_period == "m": 
                                    counter =  0
                                    time = d*(self.max_morning_periods+self.max_afternoon_periods)
                                    current_hour = i%(self.max_periods_day)
                                    mh = self.max_morning_periods - (1 if  session.group and self.rests[session.group-1] == "m" else 0)
                                    ah = self.max_afternoon_periods - (1 if session.group and  self.rests[session.group-1] == "f" else 0)
                                    while counter+time<len(time_table) and counter<mh:
                                        if self.can_add_session(session, time_table[time+counter],time+counter):
                                            time_table[time+counter].append(session)
                                            visited.append(session)
                                            break
                                        counter+=1
                            else :
                                counter =  0
                                time = d*(self.max_morning_periods+self.max_afternoon_periods)
                                while counter+time<len(time_table) and counter<self.max_morning_periods+self.max_afternoon_periods:
                                    if self.can_add_session(session, time_table[time+counter],time+counter):
                                        time_table[time+counter].append(session)
                                        visited.append(session)
                                        break
                                    counter+=1
            #####################"
            if teacher.prefered_period :
                # make this teacher's session at the period he likes if possible, else leave them till the end 
                for session in teachers_sessions[teacher] : 
                    if  session not in visited :
                        if teacher.prefered_period == "m" : #likes morning 
                            for time in range(0,len(time_table),self.max_morning_periods+self.max_afternoon_periods):
                                counter =  0
                                mh = self.max_morning_periods - (1 if session.group and self.rests[session.group-1] == "m" else 0)
                                ah = self.max_afternoon_periods - (1 if session.group and  self.rests[session.group-1] == "f" else 0)
                                while counter+time<len(time_table) and counter<mh:
                                    if self.can_add_session(session, time_table[time+counter],time+counter):
                                        time_table[time+counter].append(session)
                                        visited.append(session)
                                        break
                                    counter+=1
                                
                        elif  teacher.prefered_period == "f": 
                            for time in range(0,len(time_table),self.max_morning_periods+self.max_afternoon_periods):
                                counter = 0
                                mh = self.max_morning_periods - (1 if session.group and self.rests[session.group-1] == "m" else 0)
                                ah = self.max_afternoon_periods - (1 if session.group and  self.rests[session.group-1] == "f" else 0)
                                while counter+mh+time<len(time_table) and counter<ah:
                                    if self.can_add_session(session, time_table[time+counter+mh],time+counter+mh):
                                        time_table[time+counter+mh].append(session)
                                        visited.append(session)
                                        break
                                    counter+=1
        sessions = [session for session in sessions if session not in visited]
        if len(sessions)>0:
            return self.search_2(time_table, max_itt=10, max_itt_2=100,r=time_table,rem=sessions)
            
        return  time_table,sessions                 
                        
    def search(self,population, max_itt,max_stuck):
        soluion = []
        score = []
        index = 0
        for c,i in enumerate(population): 
            result = self.rearange(copy.deepcopy(i), max_itt,max_stuck)
            soluion.append(result)
            score.append(self.heuristic(result))
            soluion,score = mergeSort(soluion,score,"asc")
            if soluion[0] == result:
                index = c
        return soluion,score,index
    
    def neighbor_function(self,current_solution,max_itt,max_stuck):
        return self.rearange(current_solution,max_itt,max_stuck)


    def simulated_annealing(self,initial_solution, max_temp,min_temp,max_itt,max_stuck,cooling_schedule=0.01):
        current_solution = initial_solution
        current_temperature = max_temp
        energy = self.heuristic(current_solution)
        while current_temperature > min_temp:  
            neighbor_solution = self.neighbor_function(current_solution,max_itt,max_stuck)
            delta_heuristic = self.heuristic(neighbor_solution) - energy
            if delta_heuristic > 0 or random.random() < self.acceptance_probability(delta_heuristic, current_temperature):
                current_solution = neighbor_solution   
                energy = delta_heuristic
            current_temperature *= cooling_schedule
            
        return current_solution

    def acceptance_probability(self,delta, temperature):
        if delta > 0:
            return 1.0
        else:
            return np.exp(-delta / temperature)

    def heuristic(self,time_table):
        #self.priority = priority
        #self.prefered_period = prefered_period # m : morning, f:afternoon, None:anything is fine
        result = 0
        for i,period in enumerate(time_table):
            for teacher in self.teachers:
                if teacher.prefered_days: 
                    for d in teacher.prefered_days : 
                        time = d*(self.max_periods_day)
                        if i >= time and i <= time + self.max_periods_day:
                            result += 3
                if teacher.prefered_period : 
                        for session in period:
                            if session.teacher == teacher and session.group:
                                current_hour = i%(self.max_periods_day)
                                mh = self.max_morning_periods
                                ah = self.max_afternoon_periods
                                if self.rests[session.group-1] == "m":
                                    mh-=1
                                else :
                                    ah-=1
                                if (teacher.prefered_period == "m" and current_hour < mh) or (teacher.prefered_period == "f" and current_hour >= mh and current_hour < mh+ah):
                                    result += 2  # costraint teacher prefer working at morning/afternoon is met 

            ### emphasize stacking groups together can be removes 
            if len(period)>=5 : 
                result += 1
        
        return result

    def init_time_table(self):
        time_table = []
        for i in range(len(self.working_days)*(self.max_morning_periods+self.max_afternoon_periods)):
            time_table.append([])
        # randomly fill with epmty sessions befor it starts 
        for group_id in range(self.n_groups):
            for time in range(0,len(time_table),self.max_morning_periods+self.max_afternoon_periods):
                range_a = 0 if self.rests[group_id] == "m" else self.max_morning_periods
                range_b = self.max_morning_periods if self.rests[group_id] == "m" else self.max_morning_periods+self.max_afternoon_periods
                counter = random.randint(range_a,range_b)
                cool = 1000
                while time+counter>=len(time_table) and cool>0:
                    range_a = 0 if self.rests[group_id] == "m" else self.max_morning_periods
                    range_b = self.max_morning_periods if self.rests[group_id] == "m" else self.max_morning_periods+self.max_afternoon_periods
                    counter = random.randint(range_a,range_b)
                    cool-=1
                if cool == 0 and time+counter>=len(time_table):
                    counter = random.randint(time,len(time_table)-1)
                    time_table[counter].append(Session("rest", None , 1, None ,group = group_id+1,section=group_section_dict[group_id+1]))
                    continue
                time_table[time+counter].append(Session("rest", None , 1, None ,group = group_id+1,section=group_section_dict[group_id+1]))
                
        return time_table
    
    def assign_rooms(self,schedule):
        remainder = []
        for period_idx, period in enumerate(schedule):
            time = period_idx % (self.max_afternoon_periods + self.max_morning_periods)
            empty_rooms = [copy.deepcopy(r) for r in self.rooms if  r.available[time] ]
            for session in schedule[period_idx]:
                if session.session_type != "rest":
                    group = session.group
                    room = next((room for room in empty_rooms if room.room_type == session.session_type ), None)
                    if room:
                        # assigne the room to the the session
                        schedule[period_idx][schedule[period_idx].index(session)].room = room 
                        empty_rooms.remove(room) 
        for period_idx, period in enumerate(schedule):
            for session in schedule[period_idx]:
                if not session.room:
                    remainder.append(session)
        return schedule,remainder
    
    def reserve_teacher(self,timetable):
        temp = set(self.teachers)
        for i, period in enumerate(timetable):
            for session in period:
                teacher_obj = next((teacher for teacher in temp if session.teacher and teacher.id == session.teacher.id), None)
                if teacher_obj:
                    teacher_obj.available[i % (self.max_afternoon_periods + self.max_morning_periods)] = False
        self.teachers = list(temp)
    
    def reserve_rooms(self,timetable):
        temp = set(self.rooms)
        for i, period in enumerate(timetable):
            for session in period:
                room_obj = next((room for room in temp if session.room and room.id == session.room.id), None)
                if room_obj:
                    room_obj.available[i % (self.max_afternoon_periods + self.max_morning_periods)] = False
        self.rooms = list(temp)
        
    def group_has_lecture(self,group,period):
        for i in period:
            if i.session_type == "lecture" :
                if group in self.section_group_dict[i.section]:
                    return True
        return False
        
    def add_rests(self, timetable):
        reschedule = []
        for time in range(0, len(timetable), self.max_morning_periods + self.max_afternoon_periods):
            for group_id, i in enumerate(self.rests): 
                if i == "m":
                    for j in reversed(range(0, self.max_morning_periods)):
                        f = False
                       
                        if time + j < len(timetable) and not self.group_has_lecture(group_id+1,timetable[time + j]) :
                            group_exists = False 
                            for session in timetable[time + j]:
                                if session.group == group_id + 1:
                                    groyp_exists = True
                                if session.session_type !="lecture" and session.group == group_id + 1:
                                    reschedule.append(session)
                                    timetable[time + j].remove(session)
                                    timetable[time + j].append(Session("rest", None,None,1, group=session.group, section=session.section))
                                    f = True
                                    break
                            if not group_exists:
                               #def __init__(self, session_type,subject, teacher=None, length=1,section=1,times=1,group=None,per_week=1,semester=None,year=None):
                                timetable[time + j].append(Session("rest", None, None, 1, group=group_id+1, section=self.group_section_dict[group_id+1]))
                                f = True
                                break
                        if f :
                            break
                        
                elif i == "f":
                    for j in reversed(range(self.max_morning_periods, self.max_morning_periods+self.max_afternoon_periods)):
                        f = False
                        if time + j < len(timetable) and not self.group_has_lecture(group_id+1,timetable[time + j]):
                            group_exists = False 
                            for session in timetable[time + j]:
                                if session.session_type !="lecture" and session.group == group_id + 1 :
                                    reschedule.append(session)
                                    timetable[time + j].remove(session)
                                    timetable[time + j].append(Session("rest", None, None,1, group=session.group, section=session.section))
                                    f = True
                                    break
                            if not group_exists:
                                timetable[time + j].append(Session("rest", None, None,1, group=group_id+1, section=self.group_section_dict[group_id+1]))
                                f = True
                                break
                        if f :
                            break
        if len(reschedule) > 0 :
            return self.search_2(timetable, max_itt=10, max_itt_2=100, r=timetable, rem=reschedule)
        return timetable,[]
    
    def attractiveness(self,slot):
        if slot.teacher:
            return int(slot.teacher.priority) #can be optimized by some person in the field 
        else :
            return 10e5

    def calculate_world_tempuratue(self,world_temp,timetable,bird,resources_availibility):
        # emphasize leaving no empty slots between sessions ( pack )
        # emphasize teaching the same time ... pref time ... 
        # penelize those who are not valid anyway ( -inf )
        # emphasize giving lectures at morning ... other things can be added by person in the domain 
        # 1 can be changed to (weigth)
        day_hours = self.max_morning_periods+self.max_afternoon_periods
        for i, period in enumerate(timetable):
            resources = {}
            for session in period:
                if not session.session_type in resources.keys():
                    resources[session.session_type] = 0
                resources[i.session_type]+=1
            for j,session in  enumerate(period):
                temp = 0 
                if not self.can_add_session(bird,period,i):
                    temp = -1
                    world_temp[i][j] = temp 
                    continue
                else :
                    if i-1 >= 0 and i-1%day_hours<i%day_hours: 
                        for p in timetable[i-1]:
                            if p.teacher == bird.teacher or p.group == bird.group:
                                temp+=1
                    if i+1 < len(timetable) and i+1%day_hours>i%day_hours : 
                        for p in timetable[i+1]:
                            if p.teacher == bird.teacher or p.group == bird.group:
                                temp+=1
                current_time = i % day_hours
                if current_time < self.max_morning_periods and bird.teacher.prefered_period == "m":
                    temp+=1.5
                elif current_time >= self.max_morning_periods and bird.teacher.prefered_period == "f":
                    temp+=1.5
                if current_time < self.max_morning_periods and bird.session_type == "lecture":
                    temp+=.5
                if i%len(self.working_days) in bird.teacher.prefered_days:
                    temp+=2
                temp -= np.exp(resources[session.session_type]/resources_availibility[session.session_type]) # penalization 
                world_temp[i][j] = temp    
                    
        return world_temp
    
    def calculate_birds_tempuratue(self,birds_tempurature,world,resources_availibility):
        # 4.7(M/C)+32.2
        # M is the basal rate of metabolism and C is the rate of heat loss
        # M will be related to some factores such as teacher priority ( attractiveness )
        # C will be related to the world ( heuristic )
        M = self.heuristic(world) 
        for bird in birds_tempurature.keys():
            C = self.attractiveness(bird)+10e-5
            birds_tempurature[bird] -= (np.exp(4.7 * (M  / C) + 32.2) + np.exp(-1*resources_availibility[bird.session_type]))
        return birds_tempurature
    
    def calculate_resources_availibility(self,world,time_table,type):
        pass
    
    def get_resources(self,time):
        current_resources =  [copy.deepcopy(r) for r in self.rooms if  r.available[time] ]
        resources_availibility_indexer = [] 
        resources_availibility_indexer_count = {}
        for room in current_resources:
            if not room.room_type in resources_availibility_indexer_count.keys():
                resources_availibility_indexer_count[room.room_type] = 0
            resources_availibility_indexer_count[room.room_type]+=1
            resources_availibility_indexer.append(room)
        return resources_availibility_indexer,resources_availibility_indexer_count

    def migrate(self):
        time_table = []
        world_temperature = []
        remaining = []
       
        resources_availibility_indexer = [] 
        resources_availibility_indexer_count = {}
        
        for i in range(len(self.working_days)*(self.max_morning_periods+self.max_afternoon_periods)):
            time_table.append([])
            world_temperature.append([1]*self.n_groups)
        
        birds_tempurature = {}
        random.shuffle(self.sessions)
        for bird in self.sessions:
            birds_tempurature[bird] = 38
        for iteration in range(len(self.sessions)): 
            if len(birds_tempurature) == 0 :
                break
            min_bird_temp = min(birds_tempurature.values())
            cold_bird = next(key for key, value in birds_tempurature.items() if value == min_bird_temp)
            found = False
            world_temperature=self.calculate_world_tempuratue(world_temperature,time_table,cold_bird,resources_availibility_indexer_count)
            sorted_world_temperature_indexes = sorted(range(len(world_temperature)), key=lambda k: world_temperature[k], reverse=True)
            for index in sorted_world_temperature_indexes:
                if self.can_add_session(cold_bird,time_table[index],index):            
                    resources_availibility_indexer,resources_availibility_indexer_count = self.get_resources(index)      
                    #sorted_resources = sorted(resources_availibility_indexer.items(), key=lambda x: x[1], reverse=True)
                    food = [room for room in resources_availibility_indexer if bird.session_type == room.room_type]
                    if len(food) > 0:
                        cold_bird.room = food[0] # here u go, eat
                        #current_resources.pop(food[0], None)
                        resources_availibility_indexer.remove(food[0])
                        resources_availibility_indexer_count[food[0].room_type]-=1
                    time_table[index].append(cold_bird)
                    birds_tempurature.pop(cold_bird, None)
                    found = True
                    # updates
                    birds_tempurature=self.calculate_birds_tempuratue(birds_tempurature,time_table,resources_availibility_indexer_count)
                    #resources_availibility_indexer=self.calculate_resources_availibility(resources_availibility_indexer,time_table,cold)
                    break
            if not found:
                remaining.append(cold_bird)
                birds_tempurature.pop(cold_bird, None)
        # attemp to assign the rest arbitary : ( not raealy acttually ! )
        if len(remaining)>0:
            return self.search_2(time_table, max_itt=10, max_itt_2=100,r=time_table,rem=remaining)
        return  time_table,sessions   
                


In [131]:
def update_database(rooms_availability, teachers_availability):
    conn = sqlite3.connect('availability.db')
    cur = conn.cursor()

    create_rooms_table_sql = '''
    CREATE TABLE IF NOT EXISTS rooms_availability (
        room_id TEXT PRIMARY KEY,
        availability TEXT
    )
    '''
    cur.execute(create_rooms_table_sql)

    for room_id, availability_array in rooms_availability.items():
        availability_str = ','.join(map(str, [int(value) for value in availability_array]))
        cur.execute('INSERT OR REPLACE INTO rooms_availability (room_id, availability) VALUES (?, ?)', (room_id, availability_str))

    create_teachers_table_sql = '''
    CREATE TABLE IF NOT EXISTS teachers_availability (
        teacher_id TEXT PRIMARY KEY,
        availability TEXT
    )
    '''
    cur.execute(create_teachers_table_sql)

    for teacher_id, availability_array in teachers_availability.items():
        availability_str = ','.join(map(str, [int(value) for value in availability_array]))
        cur.execute('INSERT OR REPLACE INTO teachers_availability (teacher_id, availability) VALUES (?, ?)', (teacher_id, availability_str))

    conn.commit()
    conn.close()


In [132]:
def Generate(year,semester,algorithm,sessions,teachers,rooms,working_days,population_size=100,max_itt=1000,give_up=100):
    
    sessions_set  = sessions[year][semester]
    print(len(sessions_set), " session "  )
    n_sections = len(year_section_group_dict[year])
    n_groups =  sum ( [len(year_section_group_dict[year][s]) for s in year_section_group_dict[year].keys()])
    #    def __init__(self,sessions,teachers,rooms,n_sections,n_groups,working_days,max_morning_periods,max_afternoon_periods,max_periods_day,section_group_dict,perecent=50):
    generator = Generator(sessions_set,teachers,rooms,n_sections,n_groups,working_days,max_morning_periods,max_afternoon_periods,max_periods_day,year_section_group_dict[year])
    print("...Generating combinations...")
    group_conmbinations,all_unassigned_sessions = generator.populate(population_size)
    unassigned_sessions = []
    best_table,best_score=None,None
    unassigned_rooms = []
    print("...Calling The Algorithm...")
    ######
    if algorithm == 0:
        timetables,scores,i = generator.search(group_conmbinations, max_itt , give_up )
        unassigned_sessions = list(all_unassigned_sessions[i].values())
        best_score = scores[0]
        print("...Assigning Rooms...")
        timetable,unassigned_rooms = generator.assign_rooms(timetables[0])
    ###### 
    elif algorithm == 1:
        r = [[] for _ in range(generator.max_periods_day * len(generator.working_days))]
        timetable,unassigned_sessions =  generator.search_2([], max_itt=max_itt, max_itt_2=give_up,r=r,rem=sessions_set)
        print("...Assigning Rooms...")
        best_table,unassigned_rooms = generator.assign_rooms(timetable)
        best_score = generator.heuristic(best_table)
    ###### 
    elif algorithm == 2:        
        timetable,unassigned_sessions = generator.search_3()
        best_table,unassigned_rooms = generator.assign_rooms(timetable)
        print("...Assigning Rooms...")
        best_score = generator.heuristic(best_table)
    ###### 
    elif algorithm == 3:
        r = random.choice([i for i, arr in enumerate(all_unassigned_sessions) if not arr] or [-1])
        unassigned_sessions = list(all_unassigned_sessions[r].values())
        timetable =  generator.simulated_annealing(group_conmbinations[r],2,0.1,max_itt,give_up)
        print("...Assigning Rooms...")
        best_table,unassigned_rooms = generator.assign_rooms(timetable)
        best_score = generator.heuristic(best_table)
    ###### 
    elif algorithm == 4:
        timetable =  generator.migrate()
        best_score = generator.heuristic(timetable)
    
    
    print("...Adding Rest Sessions...")
    result,rem = generator.add_rests(timetable)
    unassigned_sessions+=rem
    ### reserve its resources
    generator.reserve_teacher(timetable)
    generator.reserve_rooms(timetable)
    update_database({r.id: r.available for r in generator.rooms}, {t.id: t.available for t in generator.teachers})
    ### case he wants to rollback ? ... 
    ### flash ? cache ? rollback latest update in db => flutter not the job of the Backend
    print("finished")
    return result,best_score,unassigned_sessions,unassigned_rooms


In [137]:
timetable,score,us,ur = Generate(3,2,2,sessions,list(teacher_set.values()),available_rooms,working_days,max_itt=10,give_up=1)

82  session 
...Generating combinations...
...Calling The Algorithm...
...Assigning Rooms...
...Adding Rest Sessions...
finished


In [140]:
len(timetable)

25

In [145]:

import pandas as pd

result_s2 = timetable


columns = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 5', 'Group 6']
index = pd.MultiIndex.from_product([range(1, 6), ['Morning 1', 'Morning 2', 'Morning 3', 'Afternoon 1', 'Afternoon 2','Afternoon 3']], names=['Day', 'Slot'])
df = pd.DataFrame(columns=columns, index=index)
i = 0

for period in result_s2:
        for j, session in enumerate(period):
            if session.session_type != "rest":
                if session.room:
                    teacher_subject_room =   " " + session.subject + ' ' + session.room.id
                else : 
                    teacher_subject_room = " " + session.subject + ' ' + " NO ROOM "
                if session.session_type == "lecture":
                    df.iloc[i][f'Group {1}'] = teacher_subject_room
                else :
                    df.iloc[i][f'Group {session.group}'] = teacher_subject_room
            else : 
                df.iloc[i][f'Group {session.group}'] = "REST"
        i+=1 
        
df

Group 1 Group 2 Group 3  \
Day Slot                                                                       
1   Morning 1                             Data Mining amphi8     NaN     NaN   
    Morning 2                                            NaN     NaN     NaN   
    Morning 3                                            NaN     NaN    REST   
    Afternoon 1                                          NaN     NaN     NaN   
    Afternoon 2                                          NaN     NaN     NaN   
    Afternoon 3                                         REST    REST     NaN   
2   Morning 1                                            NaN     NaN     NaN   
    Morning 2                     Operations Research amphi8     NaN     NaN   
    Morning 3                                            NaN     NaN    REST   
    Afternoon 1                                          NaN     NaN     NaN   
    Afternoon 2                                          NaN     NaN     NaN   
    Afternoon 3                                         REST    REST     NaN   
3   Morning 1                                            NaN     NaN     NaN   
    Morning 2                                            NaN     NaN     NaN   
    Morning 3                                            NaN     NaN    REST   
    Afternoon 1                                          NaN     NaN     NaN   
    Afternoon 2                                          NaN     NaN     NaN   
    Afternoon 3                                         REST    REST     NaN   
4   Morning 1     Stochastic Modelling and Simulation amphi8     NaN     NaN   
    Morning 2                                            NaN     NaN     NaN   
    Morning 3                                            NaN     NaN    REST   
    Afternoon 1                                          NaN     NaN     NaN   
    Afternoon 2                                          NaN     NaN     NaN   
    Afternoon 3                                         REST    REST     NaN   
5   Morning 1                                            NaN     NaN    REST   
    Morning 2                                            NaN     NaN     NaN   
    Morning 3                                            NaN     NaN     NaN   
    Afternoon 1                                          NaN     NaN     NaN   
    Afternoon 2                                          NaN     NaN     NaN   
    Afternoon 3                                          NaN     NaN     NaN   

                                                    Group 4  \
Day Slot                                                      
1   Morning 1                                           NaN   
    Morning 2                              Data Mining lab9   
    Morning 3                              Data Mining lab8   
    Afternoon 1                                         NaN   
    Afternoon 2                                         NaN   
    Afternoon 3                                        REST   
2   Morning 1                                           NaN   
    Morning 2                                           NaN   
    Morning 3                     Operations Research tuto7   
    Afternoon 1                                         NaN   
    Afternoon 2                                        REST   
    Afternoon 3                                         NaN   
3   Morning 1                  Networks and Protocols tuto9   
    Morning 2                                           NaN   
    Morning 3                                           NaN   
    Afternoon 1                 Networks and Protocols lab9   
    Afternoon 2                                         NaN   
    Afternoon 3                                        REST   
4   Morning 1                                           NaN   
    Morning 2     Stochastic Modelling and Simulation tuto8   
    Morning 3                                           NaN   
    Afternoon 1                            

In [142]:
def print_schedule(schedule):
    for i, period in enumerate(schedule):
        print(f"Period {i+1}:")
        for session in period:
            try:
                print(f"{session.session_type} , - Group: {session.group} - Teacher: {session.teacher.name}, Subject: {session.subject}, Room: {session.room.id}")
            except:
                print(f"{session.session_type}  {session.subject} ")
print_schedule(timetable)

Period 1:
lecture  Data Mining 
Period 2:
lab  Data Mining 
Period 3:
lab  Data Mining 
rest  None 
rest  None 
rest  None 
Period 4:
lab  Data Mining 
Period 5:
lab  Data Mining 
Period 6:
lab  Data Mining 
rest  None 
rest  None 
rest  None 
Period 7:
lab  Data Mining 
Period 8:
lecture  Operations Research 
Period 9:
tuto  Operations Research 
rest  None 
rest  None 
rest  None 
Period 10:
tuto  Operations Research 
Period 11:
tuto  Operations Research 
rest  None 
Period 12:
lecture  Networks and Protocols 
rest  None 
rest  None 
Period 13:
tuto  Networks and Protocols 
Period 14:
tuto  Networks and Protocols 
Period 15:
rest  None 
rest  None 
rest  None 
rest  None 
Period 16:
lab  Networks and Protocols 
Period 17:
lab  Networks and Protocols 
Period 18:
lab  Networks and Protocols 
rest  None 
rest  None 
rest  None 
Period 19:
lecture  Stochastic Modelling and Simulation 
Period 20:
tuto  Stochastic Modelling and Simulation 
Period 21:
rest  None 
rest  None 
rest  None 
rest

In [33]:
### radwa 
# add propagation on rooms 
# add birds algo 
# table 
### scurity cours 
### exo ai
### numirique cours


In [ ]:
"""



whats something is the nature, bodies of animals or human .. ect or an animal or a pecies or anything that can "schedual" something depeding on its need
ChatGPT
In nature, many organisms exhibit behaviors or processes that can be considered as scheduling based on their needs. One prominent example is the migration patterns of certain animals. Animals such as birds, whales, and butterflies often follow intricate schedules in their migrations, timing their journeys based on factors like weather conditions, availability of food, or breeding cycles.

For instance, birds might migrate thousands of miles to warmer climates during winter months to ensure access to food and suitable breeding grounds. The timing of this migration is crucial, and it's often finely tuned to environmental cues such as changes in daylight hours.

Similarly, many animals exhibit daily rhythms or circadian rhythms, which involve scheduling activities such as feeding, resting, and hunting according to the time of day. These rhythms are often influenced by internal biological clocks as well as external factors like sunlight.

In the case of humans, our lives are heavily scheduled based on various needs, such as work, sleep, and social activities. While much of this scheduling is influenced by societal norms and obligations, our bodies also have internal rhythms that influence when we feel most alert, when we're most likely to sleep, and when we experience peak levels of productivity.

Overall, scheduling behaviors in nature can be observed across a wide range of organisms, and they often serve to optimize survival and reproductive success in dynamic environments.

suppose you want to create an algorithm for scheduling prblem, how can u take adcantage of this ?
ChatGPT
In creating an algorithm for scheduling, whether it's for tasks, resources, or events, leveraging insights from scheduling behaviors in nature can be highly advantageous. Here are some ways you could integrate these insights into your algorithm:

Adaptability: Just like migratory patterns adapt to changing environmental conditions, your algorithm could incorporate adaptability. It could dynamically adjust schedules based on real-time changes in factors like resource availability, task priorities, or unexpected events.

Optimization: Many animals optimize their schedules to maximize benefits like food intake or reproductive success. Your algorithm could use optimization techniques such as genetic algorithms, simulated annealing, or ant colony optimization to find the most efficient schedules based on specified criteria.

Rhythms and Patterns: Incorporate the concept of rhythms and patterns into your algorithm. Humans and many animals have daily, seasonal, or even longer-term rhythms that influence their activities. By considering these patterns, your algorithm can schedule tasks or events at times when they're most likely to be successful or efficient.

Risk Management: Animals often balance risks and rewards when scheduling activities like foraging or migration. Your algorithm could consider risk management strategies, such as scheduling buffer times to account for uncertainties or avoiding scheduling critical tasks during periods of high risk.

Hierarchical Scheduling: Many animals engage in hierarchical scheduling, where they prioritize tasks based on their importance or urgency. Your algorithm could incorporate hierarchical scheduling techniques to efficiently allocate resources and prioritize tasks based on their significance.

Learning and Adaptation: Animals learn from experience and adapt their schedules accordingly. Your algorithm could incorporate machine learning or reinforcement learning techniques to continuously improve scheduling decisions based on past outcomes and feedback.

By integrating these insights from scheduling behaviors in nature, your algorithm can become more robust, adaptable, and efficient in solving various scheduling problems across different domains.







"""


